# EOF analysis and PC regression
Notebook to do the complete analysis of:
* selecting a subset of data (e.g. winter months)
* EOF analysis
* PC regression

for a number of different model data, as well as 
* comparing the analysis results of the different models visually and statistically with Taylor diagrams

Required input: pre-processed data (e.g. with CDO):
* required years
* required geographical domain
* de-trended
* converted to one grid

REQUIRED FILE STRUCTURE:
* Folder 'Functions' with external functions
* Folder 'data' with subfolders for each model containing files for geopotential height, temperature and precipitation
    * named 'region_variable_tempresolution_model_scenario_variant_grid_timeframe_detrended_grid.nc'
    * e.g. 'NA_zg_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412_detrended_regriddedtoIPSL.nc' 
* Folder 'output' with subfolders for each model, as well as subfolders 'plots', 'taylordiagrams', 'mean_diff', 'PCRplots' for model comparison

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

### Import required packages and external functions:

In [ ]:
%load_ext autoreload

In [ ]:
from netCDF4 import Dataset, num2date
import matplotlib as mpl
from matplotlib import gridspec,cm, rcParams, pyplot as plt
import numpy as np
import pandas as pd
import math
import scipy
from scipy import stats
from sklearn.linear_model import LinearRegression
from eofs.standard import Eof
from eofs.tools.standard import correlation_map, covariance_map
from os.path import exists
import sys  
from uncertainties import ufloat, unumpy
# mapping:
from cartopy import config, crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import skill_metrics as sm


# import my own functions:
# (stored in the same location as this Notebook in a folder "Functions")
sys.path.append('Functions')
from data_selection import subset_data, select_extreme_quartiles, find_month_from_index, select_extreme_months
from EOF_analysis import EOF_analysis, plot_mean_field_func, plot_EOFs, plot_PCs
from PC_regression import PC_regression, plot_PCR, plot_mean_field
from naming_functions import name_files, plot_titles
from detrending import detrend_files
from plot_extremes import select_extremes, plot_extremes_in_PCs, plot_mean_fields_of_extr_months
from plotting_functions import plot_map, plot_PCR1, plot_meandiffs, plot_eigenvalues, plot_1_PCR1
from model_comparison import plot_mean_plus_diff, plot_diff_map, plot_diff_p_map, plot_4_diff_map, plot_EOFs_plus_gradlocs, plot_EOF1_plus_gradlocs  
from model_comparison import calc_gradients, test_gradient_overlap, z_test_two_means, plot_mean_diff_ranking, plot_gradient_ranking
from model_comparison2 import taylor_metrics, GL_taylor_metrics, skill_score, plot_taylor_diagrams, plot_1_taylor_diagram, weighted_skillscore


%autoreload 2


### Define variables for the used data:

In [ ]:

models = ['ERA5', 'CESM2', 'CNRM-ESM2-1', 'EC-Earth3 r1', 'EC-Earth3 r10', 'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'UKESM1-0-LL']#, 'EC-Earth3', 'EC-Earth3']
variants = ['','r1i1p1f1', 'r1i1p1f2',   'r1i1p1f1',     'r10i1p1f1',   'r1i1p1f1',   'r1i1p1f1',  'r1i1p1f1',     'r1i1p1f2']   


types = {1:'allmonths',2:'monthly',3:'seasonal'}
seasons = {'winter':'DJF', 'spring':'MAM', 'summer':'JJA', 'fall':'SON','extended cold':'NDJFM'}
months = {1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
regtypes = ['regression','correlation', 'covariance']
regdatatypes = ['Geopotential height','Precipitation', 'Temperature']
regdatatypes_short = {'Geopotential height':'zgs','Precipitation':'pr', 'Temperature':'tas'}
regdatatypes_units = {'Geopotential height':'[m]','Precipitation':'[mm/day]', 'Temperature':'[°C]'}


# swap EOFs if necessary to compare same patterns (e.g. EA with EA):
orders = np.array([[0,1,2,3], # ERA5
                  [0,1,3,2], # CESM2
                  [0,1,3,2], # CNRM
                  [0,2,1,3], # EC-Earth r1
                  [0,1,2,3], # EC-Earth r10
                  [0,1,2,3], # IPSL
                  [0,1,3,2], # MIROC6 
                  [0,1,2,3], # MPI
                  [0,2,3,1]  # UKESM
                  #[0,2,1,3]  # EC-Earth r101 
                  ])



### Define analysis settings:

In [ ]:

height_level = 5                         # 5 = geopotential height at 500 hPa
analysis_type = types[3]                 # 3 = seasonal
season = seasons['extended cold']
season_mean = True
running_mean = 1                         # if 1, no running mean, just seasonal mean is used
month = months[1]                        # only relevant if analysis_type = monthly
reg_type = regtypes[0]                   # 0 = regression
regdata_type = regdatatypes[2]
regridded_text='_regriddedtoIPSL'

# optional: look at extremes in sub-region (Greenland):
extremes=False
extr_type = 'invalid'
extreme_region = 'Gl_masked'#'NE' 
extreme_region_name = 'Greenland' #if (extreme_region=='Gl')
nmonths = 5 # how many extreme months to look at (at least 5)

startyear, endyear = 1959, 2014
timeframe_long, timeframe = f'{startyear}01-{endyear}12', f'{startyear}-{endyear}' 

n_eofs = 4
norm_PC = False                  # normalize PCs by their sum per month, to show relative importance of the leading 4?

# plotting:
n_colorlevels = 11               # colorlevels for plotting
plot_mean_field = True
add_eof_pattern = True
mark_high_corr=False
mask_low_corr=False #True
corr_threshold=0.3               # threshold to consider correlation coefficient 'high' or robust
fixed_bounds= True #False        # use prescribed limits to standardize colorscale for all regression plots (see Functions>PC_regression.py)

savemean = True
saveEOF = True 
savePC = True 
savePCreg = True
savemetrics = True
Greenland_analysis=False

regression_loop = [0,1,2]        # Do geopotential height, temperature and precipitation regression 
# (numbers refer to 'regdatatypes' above)


comp_dims = 21 
# size of the arrays containing the comparison results
# 0 = mean zg field, 1 - 5 = eofs 1 - 4, 6 - 9 = PCR 1 -4, 10 - 13 = high corr PCR 1-4, 14-21 GL PCR
ccoef = np.zeros((comp_dims,len(models)))
sdev = np.zeros((comp_dims,len(models)))
mse = np.zeros((comp_dims,len(models)))
rmse = np.zeros((comp_dims,len(models)))

mean_diff_arr = np.zeros((2,len(models)))   # normal and significant differences
NAO_grad = np.zeros((2,len(models)))




## EOF & PCR execution:
DO EOF analysis and PC regression for each model and plot the results

In [ ]:
%autoreload 2

savedata = True
loaddata = False #True #False    # if True loads EOFs/PCs and regression coefficients that have been calculated and saved in previous runs to save computation time


regression_loop = [0,1,2]        # Do geopotential height, temperature and precipitation regression 
# (numbers refer to 'regdatatypes' above)

for regressiontype in regression_loop:    
    regdata_type = regdatatypes[regressiontype] 
    print(f'--- {regdata_type} Regression ---')

    for modeli in range(len(models)):
        model = models[modeli]
        print(model)
        variant = variants[modeli]
        order = orders[modeli]
        #grid = 'gr' if (modeli in [0,1,2,3,8,9]) else 'gn'
        grid = 'gr' if (modeli in [0,2,3,4,5]) else 'gn'               # check that this is consistent with the naming in the used model data
        timeframe = f'{startyear}-{endyear}'    
        if (model=='ERA5'):
            eofvar = 'z' 
            regvar = 'tp' if (regdata_type=='Precipitation') else 't2m'
        else:
            eofvar = 'zg'
            regvar = 'pr' if (regdata_type=='Precipitation') else 'tas'
        if (regdata_type=='Geopotential height'):
            regvar = eofvar 



        if (model=='ERA5'):
            startyear, endyear = 1959, 2020
        else:
            startyear, endyear = 1850, 2014  
        timeframe, timeframe_long = f'{startyear}-{endyear}' ,  f'{startyear}01-{endyear}12'
        # input files:
        base_filename = f'Amon_{model}_historical_{variant}_{grid}_{timeframe_long}'
        eof_infile_detrended_nc = f'data/{model}/NA_{eofvar}_{base_filename}_detrended{regridded_text}.nc'
        regression_infile = f'data/{model}/NA_{regvar}_{base_filename}_detrended{regridded_text}.nc' 

        dataset = Dataset(eof_infile_detrended_nc)
        lats, lons = dataset.variables['lat'][:] , dataset.variables['lon'][:] 
        zgs_detrended = dataset.variables[eofvar] 
        if (model=='ERA5'):
            zgs = zgs_detrended[:] / 9.80665             # to turn geopotential into geopotential height
        elif (len(np.shape(zgs_detrended))>3):             # if variable has several height levels
            print(f' Shape original data: {np.shape(zgs_detrended)}')
            zgs = zgs_detrended[:,height_level, :, :]    # select geopotential height at 500 hPa (= 5000 Pa)
        else:
            zgs = zgs_detrended 
        model_zg_data = subset_data(zgs, analysis_type, season, season_mean, month, running_mean)
        #model_zg_mean = np.mean(model_zg_data, axis=0)

        # load regression data:
        if (regdata_type=='Geopotential height'):
            model_reg_data = model_zg_data
        else:
            reg_dataset = Dataset(regression_infile)
            reg_data_raw = reg_dataset.variables[regvar] 
            model_reg_data = subset_data(reg_data_raw, analysis_type, season, season_mean, month, running_mean)
            if (regdata_type=='Temperature'):  # transform from K to °C
                model_reg_data -= 273.15
            if (regdata_type=='Precipitation'):  # transform to mm/day
                if (model=='ERA5'):              # from m/day 
                    model_reg_data *= 1000
                else:                            # from kg/m2/s
                    model_reg_data *= 86400

        model_regdata_mean = np.mean(model_reg_data, axis=0)

        if (len(model_reg_data)==55):
            startyear, endyear = 1959, 2014
            timeframe_long = f'{startyear}01-{endyear}12'
            timeframe = f'{startyear}-{endyear}' 


        # EOF & PCR: ------------------------
        ID, savemeanas, saveEOFas, savePCas, savePCregas, saveregmeanas, saveEOFdataas, savePCRdataas, saveextremesplotas, saveextremesmapas = name_files(analysis_type,month,season,season_mean,running_mean,extremes,extr_type,extreme_region,nmonths,timeframe,model,variant,n_eofs,regdatatypes_short,regdata_type,reg_type,mask_low_corr,regridded_text)
        standard_title, base_title = plot_titles(season_mean,running_mean,extremes,extr_type,extreme_region_name,model,variant,season,timeframe)

        model_eofs, model_pcs, model_varfrac = EOF_analysis(model_zg_data, lats, lons, n_eofs, analysis_type, season, season_mean, 
                                                            timeframe, model, variant, ID, running_mean, extremes, startyear, endyear, standard_title,
                                                            savedata, saveEOFdataas, loaddata, savemean, savemeanas, saveEOF, saveEOFas, norm_PC, savePC, savePCas, n_colorlevels, plot_mean_field=True,order=order)

        model_coefficients, model_corrcoeffs = PC_regression(model_reg_data, reg_type, regdata_type, regdatatypes_units, lats, lons, model_pcs, n_colorlevels, model_eofs, model_varfrac,
                                                             loaddata, savedata, savePCRdataas, model, saveregmeanas,
                                                             add_eof_pattern, mark_high_corr, mask_low_corr, corr_threshold, standard_title, savePCreg, savePCregas, fixed_bounds, order)






## Model comparison
- Calculate the metrics to copmare the results from each model against each other
- Plot difference plots and Taylor diagrams

In [ ]:
%autoreload 2


loaddata = False     # if True loads coefficients that have been calculated and saved in previous runs to save computation time



# --------------------------------------------------------------------------------------------------
regression_loop = [0,1,2]

for regressiontype in regression_loop:   
    regdata_type = regdatatypes[regressiontype] 
    print(f'--- {regdata_type} Regression ---')

    # load ERA5 data as base for comparison:
    # --------------------------------------------------
    model = models[0]
    variant = variants[0]
    order = orders[0]  
    eofvar = 'z' 
    regvar = 'tp' if (regdata_type=='Precipitation') else 't2m'
    if (regdata_type=='Geopotential height'):
        regvar = eofvar 
    season = 'NDJFM'
    meantext = 'mean1'
    extr_text = ''

    obs_zg_mean = np.load(f'data/{model}/meanzg_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_reg_data = np.load(f'data/{model}/reg_data_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_regdata_mean = np.load(f'data/{model}/meanfield_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_eofs = np.load(f'data/{model}/eofs_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_pcs = np.load(f'data/{model}/pcs_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_varfrac = np.load(f'data/{model}/varfrac_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_coefficients = np.load(f'data/{model}/coeffs_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
    obs_corrcoeffs = np.load(f'data/{model}/corrcoeffs_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')


    obs_file = f'data/ERA5/NA_z_Amon_ERA5_historical_1_gr_195901-202012_detrended{regridded_text}.nc'
    dataset = Dataset(obs_file)
    lats, lons = dataset.variables['lat'][:] , dataset.variables['lon'][:] 
  

    

##### load model data to compare with ERA5:
    # ----------------------------------------------------
    for modeli in range(len(models)):
        model = models[modeli]
        print(model)
        variant = variants[modeli]
        order = orders[modeli]
        grid = 'gr' if (modeli in [0,2,3,4,5]) else 'gn'  
        if (model=='ERA5'):
            eofvar = 'z' 
            regvar = 'tp' if (regdata_type=='Precipitation') else 't2m'
        else:
            eofvar = 'zg'
            regvar = 'pr' if (regdata_type=='Precipitation') else 'tas'
        if (regdata_type=='Geopotential height'):
            regvar = eofvar 
         
        
        model_zg_mean = np.load(f'data/{model}/meanzg_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_eofs = np.load(f'data/{model}/eofs_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_pcs = np.load(f'data/{model}/pcs_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_varfrac = np.load(f'data/{model}/varfrac_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_reg_data = np.load(f'data/{model}/reg_data_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_regdata_mean = np.load(f'data/{model}/meanfield_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_coefficients = np.load(f'data/{model}/coeffs_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')
        model_corrcoeffs = np.load(f'data/{model}/corrcoeffs_PC_{regdatatypes_short[regdata_type]}_regression_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')

        
        
        
# COMPARISON ANALYSIS:
# ---------------------------------------------------  
        standard_title, base_title = plot_titles(season_mean,running_mean,extremes,extr_type,extreme_region_name,model,variant,season,timeframe)
        
        latweights = np.array(np.cos(np.deg2rad(lats)))  
        area_weights = np.tile(latweights, (len(lons), 1)).T


        PCR1fig = plot_1_PCR1(regdata_type, model_eofs, model_varfrac, model_coefficients, model_corrcoeffs, n_colorlevels, lons, lats, regdatatypes_units, standard_title)
        savePCR1figas = f'output/{model}/SinglePCR1plot_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        PCR1fig.savefig(savePCR1figas)
        
        
        # plot difference plots:
        # ---------------------------------------------------
        
        
        # CALCULATE DIFFERENCE:
        diff_mean = model_regdata_mean - obs_regdata_mean 
        try:
            len(diff_mean.flat)
        except TypeError:
            len(diff_mean.data.flat)
            diff_mean = diff_mean.data
            
        # spatial mean of differences:
        diff_mean_mean = np.average(diff_mean, weights=area_weights)
        mean_diff_arr[0,modeli] = diff_mean_mean
        
        #print(f'Mean difference = {diff_mean_mean:.2f}{regdatatypes_units[regdata_type]}')
        #print(f'Highest difference = {np.max(diff_mean):.2f}{regdatatypes_units[regdata_type]}')
        #print(f'Lowest difference = {np.min(diff_mean):.2f}{regdatatypes_units[regdata_type]}')
        
        # calculate & plot p-value of means comparison:
        diff_mean_p = z_test_two_means(model_reg_data,obs_reg_data)
        diff_title= f'Prob (p-value) of similarity to ERA5 mean {regdata_type} field'
        diff_text = f'Mean difference: {diff_mean_mean:.2f} {regdatatypes_units[regdata_type]}'
        diff_plot = plot_diff_p_map(diff_mean_p, lons, lats, regdata_type, fixed_bounds, standard_title, diff_title, diff_text)
        savediffmeanas = f'output/{model}/meandifftoERA5_probabilityp_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        diff_plot.savefig(savediffmeanas)
        
        # significant difference:
        diff_mean_masked = np.ma.masked_where(diff_mean_p > 0.05, diff_mean)
        diff_mean_masked_mean = np.average(diff_mean_masked, weights=area_weights)
        mean_diff_arr[1,modeli] = diff_mean_masked_mean
        print('Significant differences (p<0.05): ')
        #try:
        print(f'{diff_mean_masked.count()} / {len(diff_mean.flat)} = {(diff_mean_masked.count() / len(diff_mean.flat))*100:.2f} %' )
        #except AttributeError:
            #print(f'{diff_mean_masked.count()} / {len(diff_mean.data.flat)} = {(diff_mean_masked.count() / len(diff_mean.data.flat))*100:.2f} %' )
        
        print(f'Mean difference = {diff_mean_masked_mean:.2f}{regdatatypes_units[regdata_type]}')
        frac_diff = diff_mean_masked.count() / len(diff_mean.flat)
        print(f'Mean diff * frac diff = {diff_mean_masked_mean} * {frac_diff} = {diff_mean_masked_mean * frac_diff}')
        #print(f'Highest difference = {np.max(diff_mean_masked):.2f}{regdatatypes_units[regdata_type]}')
        #print(f'Lowest difference = {np.min(diff_mean_masked):.2f}{regdatatypes_units[regdata_type]}')
        
        plotting_modes=['mean+diff','mean+era','diff+era']
        plotting_mode = plotting_modes[2]
        diff_text= f'Mean difference: {diff_mean_mean:.2f} {regdatatypes_units[regdata_type]}'
        mean_diff_plot = plot_mean_plus_diff(model_regdata_mean,diff_mean,obs_regdata_mean,lons,lats,plotting_mode,regdata_type,standard_title,diff_text)
        savediffmeanas = f'output/{model}/meanpattern_{plotting_mode}_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        mean_diff_plot.savefig(savediffmeanas)
        
        diff_text= f'Mean difference: {diff_mean_masked_mean:.2f} {regdatatypes_units[regdata_type]}'
        mean_diff_plot = plot_mean_plus_diff(model_regdata_mean,diff_mean_masked,obs_regdata_mean,lons,lats,plotting_mode,regdata_type,standard_title,diff_text,masked=True)
        savediffmeanas = f'output/{model}/meanpattern_{plotting_mode}_masked_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        mean_diff_plot.savefig(savediffmeanas)
        
        '''
        pcr_diff = obs_coefficients-model_coefficients
        diff_title= f'{regdata_type} regression \n Difference ERA5 - {model} \n Mean difference: {np.mean(pcr_diff):.2f} {regdatatypes_units[regdata_type]}'
        diff_plot = plot_4_diff_map(pcr_diff, lons, lats, model_eofs, regdata_type, fixed_bounds, diff_title, order)
        savediffplotas = f'output/{model}/PCR_difftoERA5_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        diff_plot.savefig(savediffplotas)      
        '''

        # Calculate comparison metrics:
        # ---------------------------------------------------
        # mean field gradients:
        obs_grads = np.array(calc_gradients(obs_reg_data, lats, lons))
        model_grads = np.array(calc_gradients(model_reg_data, lats, lons))
        model_rel_grads = (model_grads-obs_grads)/obs_grads
        grad_agreement = test_gradient_overlap(obs_grads,model_grads)
        NAO_grad[0,modeli] = model_grads[0].nominal_value
        NAO_grad[1,modeli] = grad_agreement[0]
        #NAO_grad[2,modeli] = model_grads[0].nominal_value-obs_grads[0].nominal_value
        print('Z500 gradients, Within 95% CI of ERA5, Difference to ERA5, Rel diff to ERA5 ')
        for igrad in range(4):
            print(f'{model_grads[igrad]:.2f}, {grad_agreement[igrad]:.2f}, {model_grads[igrad]-obs_grads[igrad]:.2f}, {model_rel_grads[igrad]:.2f}')
        #print(f'Z500 gradients: {model_grads}')
        #print(f'Within 95% CI of ERA5: {grad_agreement}')
        #print(f'Difference to ERA5: {model_grads-obs_grads}')
        #print(f'Relative difference to ERA5: {model_rel_grads}')
        #print(f'Z500 gradients: {calc_gradients(model_zg_mean, lats, lons)}')
        
        
        ccoef, sdev, mse, rmse = taylor_metrics(ccoef,sdev,mse,rmse,obs_regdata_mean,model_regdata_mean,
                                                obs_eofs,model_eofs,obs_coefficients,model_coefficients,
                                                obs_corrcoeffs,model_corrcoeffs,
                                                corr_threshold,area_weights,modeli,loaddata,models)
        
        
        if Greenland_analysis:
        # Greenland focus:
        #Greenland: 63-13°W / 59-84°N 
            model_high_corr_reg_coefficients = np.where(np.logical_or(model_corrcoeffs > corr_threshold, model_corrcoeffs<-corr_threshold), model_coefficients, np.nan)
            obs_high_corr_reg_coefficients = np.where(np.logical_or(obs_corrcoeffs > corr_threshold, obs_corrcoeffs<-corr_threshold), obs_coefficients, np.nan)


            GL_lat1, GL_lat2 = np.absolute(lats-59).argmin(), np.absolute(lats-84).argmin()
            GL_lon1, GL_lon2 = np.absolute(lons-(-63)).argmin(), np.absolute(lons-(-13)).argmin()
            GL_model_coefficients = model_coefficients[:,GL_lat1:GL_lat2,GL_lon1:GL_lon2]
            GL_obs_coefficients = obs_coefficients[:,GL_lat1:GL_lat2,GL_lon1:GL_lon2]
            GL_lons=lons[GL_lon1:GL_lon2]
            GL_lats=lats[GL_lat1:GL_lat2]
            GL_model_high_corr_reg_coefficients=model_high_corr_reg_coefficients[:,GL_lat1:GL_lat2,GL_lon1:GL_lon2]
            GL_obs_high_corr_reg_coefficients=obs_high_corr_reg_coefficients[:,GL_lat1:GL_lat2,GL_lon1:GL_lon2]
            GL_model_eofs=model_eofs[:,GL_lat1:GL_lat2,GL_lon1:GL_lon2]
            GL_latweights = np.array(np.cos(np.deg2rad(GL_lats)))  
            GL_area_weights = np.tile(GL_latweights, (len(GL_lons), 1)).T
            if mask_low_corr:
                GL_savePCregas=f'output/{model}/PC_{regdatatypes_short[regdata_type]}_{reg_type}_GL_masked_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}{regridded_text}.pdf'
            else:
                GL_savePCregas=f'output/{model}/PC_{regdatatypes_short[regdata_type]}_{reg_type}_GL_full_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}{regridded_text}.pdf'
            plot_PCR(fixed_bounds,regdata_type,regdatatypes_units,reg_type,n_colorlevels,GL_model_coefficients,
                     GL_lons,GL_lats,
                     add_eof_pattern,mark_high_corr,mask_low_corr,
                     GL_model_high_corr_reg_coefficients,GL_model_eofs,
                     corr_threshold,model_varfrac,standard_title,GL_savePCregas,savePCreg,order)


            ccoef, sdev, mse, rmse= GL_taylor_metrics(ccoef,sdev,mse,rmse,GL_obs_coefficients,GL_model_coefficients,
                                                      GL_obs_high_corr_reg_coefficients,GL_model_high_corr_reg_coefficients,
                                                      GL_area_weights,modeli,loaddata,models)


    compmetrics = np.array([ccoef,sdev,mse,rmse])
    
    skillscore = plot_taylor_diagrams(compmetrics, regvar, models, regdata_type)
                                

    weighted_score=weighted_skillscore(skillscore,models,timeframe,variants,season,meantext,regridded_text,
                                       regvar,regdata_type,obs_varfrac)   
    
    plot_mean_diff_ranking(mean_diff_arr, regvar, models, regdata_type, regdatatypes_units)
    plot_meandiffs(lons, lats, mean_diff_arr, models, variants, orders, regdata_type, regdatatypes_short, regdatatypes_units)

    if (regvar=='zg'):
        plot_gradient_ranking(NAO_grad, models)
    
    if Greenland_analysis:
        GL_skillscore = plot_taylor_diagrams(compmetrics, regvar, models, regdata_type, area='Greenland')

        GL_weighted_score=weighted_skillscore(GL_skillscore,models,timeframe,variants,season,meantext,regridded_text,
                                           regvar,regdata_type,obs_varfrac,area='Greenland')    


    
    #compmetrics = np.array([ccoef,sdev,mse,rmse,skillscore])
    
    # save comparison metrics:
    if savemetrics:
        #compmetrics = np.array([ccoef,sdev,mse,rmse])
        np.save(f'data/compmetrics_{regvar}',compmetrics)
        np.save(f'data/skillscore_{regvar}',skillscore)
        np.save(f'data/weightedskillscore_{regvar}',weighted_score)
        np.save(f'data/mean_diff_{regvar}', mean_diff_arr)
                                  
            
            
    
    # plot PCR and Taylor diagram for each PC seperately:
    for PCnum in range(4): # 0 for NAO
        plot_PCR1(mask_low_corr, add_eof_pattern, corr_threshold, lons, lats, models, variants, orders, 
                  regdata_type, regdatatypes_short, regdatatypes_units, reg_type, n_colorlevels, PCnum, 
                  fixed_bounds=True, savefig=True)

        plot_1_taylor_diagram(compmetrics, regvar, models, regdata_type, PCnum, area='')
    
    

    

    
plot_eigenvalues(models, variants)




## Further analysis and plotting figures (optional)
This is just a collection of code to make some further analyses and figures

In [ ]:
# Find the exact locations of the min and max values of the regression:

maxreg = np.argmax(model_coefficients[0,:])
model_coefficients[0,:].flat[maxreg]
#lats.flat[maxreg]


PCnum = 0
maxreg = np.argmax(model_coefficients[PCnum,:])
ind = np.unravel_index(maxreg, (len(lats),len(lons)))
print(f'Max PC {PCnum+1} {regdata_type} regression value: {model_coefficients[PCnum,:].flat[maxreg]} at lat: {lats[ind[0]]} , lon: {lons[ind[1]]} ')

minreg = np.argmin(model_coefficients[PCnum,:])
ind = np.unravel_index(minreg, (len(lats),len(lons)))
print(f'Min PC {PCnum+1} {regdata_type} regression value: {model_coefficients[PCnum,:].flat[minreg]} at lat: {lats[ind[0]]} , lon: {lons[ind[1]]} ')


lats, lons, ind

In [ ]:
# Figure to illustrate the principle of the PC regression:

maxreg = np.argmax(model_coefficients[PCnum,:])
ind = np.unravel_index(maxreg, (len(lats),len(lons)))
print(f'Max PC {PCnum+1} {regdata_type} regression value: {model_coefficients[PCnum,:].flat[maxreg]} at lat: {lats[ind[0]]} , lon: {lons[ind[1]]} ')

#minreg = np.argmin(model_coefficients[PCnum,:])
#ind = np.unravel_index(minreg, (len(lats),len(lons)))
#print(f'Min PC {PCnum+1} {regdata_type} regression value: {model_coefficients[PCnum,:].flat[minreg]} at lat: {lats[ind[0]]} , lon: {lons[ind[1]]} ')


col_mean = np.mean(model_reg_data, axis=0)
anoms = model_reg_data - col_mean
anomaly_ts = anoms[:,ind[0],ind[1]]

#anomaly_ts = model_reg_data[:,ind[0],ind[1]] - np.mean(model_reg_data[:,ind[0],ind[1]], axis=0)
pc1 = model_pcs[:,0]



plt.style.use('default')
fig_PC, axs = plt.subplots(2, 1, figsize=(5,5))#, sharex=True)
tickmdist = 10  # one tick mark every 10 years

ax = axs[0]
clrs = ['red' if (x > 0) else 'blue' for x in pc1 ]
myxaxis = np.arange(len(pc1 ))
ax.bar(myxaxis, pc1 , color=clrs)
myticks = np.arange(0,len(myxaxis),tickmdist);
mylabels = np.arange(startyear,endyear+1,tickmdist) 

ax.set_xticks(ticks=myticks); 
ax.set_xticklabels(labels=mylabels);
ax.set_xlabel('Model Years');

ax.margins(x=0.005, y=0)
ax.set_title(f'PC1 time series', loc='left', fontsize='xx-large');
ax.set_ylabel('Standard deviations')


ax = axs[1]
clrs = ['red' if (x > 0) else 'blue' for x in anomaly_ts]
myxaxis = np.arange(len(anomaly_ts))
ax.bar(myxaxis, anomaly_ts, color=clrs)
myticks = np.arange(0,len(myxaxis),tickmdist);
mylabels = np.arange(startyear,endyear+1,tickmdist) 

ax.set_xticks(ticks=myticks); 
ax.set_xticklabels(labels=mylabels);
ax.set_xlabel('Model Years');

ax.margins(x=0.005, y=0)
ax.set_title(f'Temperature anomalies', loc='left', fontsize='xx-large');
ax.set_ylabel('Temperature [°C]')
#fig_PC.suptitle(standard_title)
fig_PC.tight_layout()
fig_PC.savefig('output/PCR1.pdf')

fig, ax = plt.subplots(figsize=(4,3))
ax.scatter(anomaly_ts,pc1,color='black')
ax.set(xlabel='Temperature anomalies',ylabel='EOF1 loading')

reg = LinearRegression().fit(pc1.reshape(-1, 1), anomaly_ts.reshape(-1, 1))
r = reg.coef_[0,0]
R = np.corrcoef(pc1,anomaly_ts)[0,1]
print(r,R)

ax.text(-4,2,f'R = {R:.2f}',fontsize='large')
ax.text(-4,1.5,f'r = {r:.2f}',fontsize='large')
fig.tight_layout()
fig.savefig('output/PCR2.pdf')

In [ ]:
gradfig = plot_EOF1_plus_gradlocs(model_eofs,n_colorlevels,lons,lats,model_varfrac,standard_title,order);



In [ ]:
pattern = ['NAO', 'AR', 'SB', 'WE']
for ieof in range(4):
    #ieof = 1
    compbase = 'highcorr'  # = +4
    ss_zg = np.load('data/skillscore_zg.npy')[ieof+4]
    ss_tas = np.load('data/skillscore_tas.npy')[ieof+4]
    ss_pr = np.load('data/skillscore_pr.npy')[ieof+4]

    ss_o = (ss_zg + ss_tas + ss_pr)/3

    cell_text=[]
    for i in range(len(models)):
        if (i==0):
            continue
        index = np.argsort(ss_o)[::-1][i];
        cell_text.append([models[index], np.round(ss_o[index],3)])


    fig, ax = plt.subplots(figsize=(3,5)) ; #, sharey=True);
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=cell_text, cellLoc='left', colLabels=['Model','Skill Score'],loc='center',bbox = [0, 0, 1, 1])
    #ax.set_title(f'Combined PC {ieof+1} Regression Score', loc='left', fontsize='small')
    #fig.suptitle(f'Combined PC {ieof+1} Regression Score', fontsize='small')
    fig.suptitle(f'Combined {pattern[ieof]} Score', fontsize='large')
    fig.tight_layout();
    fig.savefig(f'output/taylordiagrams/taylor_scoretables_PC{ieof+1}_allvars_{compbase}.pdf') # {timeframe}
    fig.savefig(f'output/plots/taylor_scoretables_PC{ieof+1}_allvars_{compbase}.pdf') # {timeframe}



    ss_o = (ss_tas + ss_pr)/2

    cell_text=[]
    for i in range(len(models)):
        if (i==0):
            continue
        index = np.argsort(ss_o)[::-1][i];
        cell_text.append([models[index], np.round(ss_o[index],3)])


    fig, ax = plt.subplots(figsize=(3,7)) ; #, sharey=True);
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=cell_text, cellLoc='left', colLabels=['Model','Skill Score'],loc='center',bbox = [0, 0, 1, 1])
    #ax.set_title(f'Combined PC {ieof+1} Regression Score', loc='left', fontsize='small')
    fig.suptitle(f'Combined PC {ieof+1} Temperature & Precipitation Regression Score', fontsize='xx-small')
    fig.tight_layout();
    fig.savefig(f'output/taylordiagrams/taylor_scoretables_PC{ieof+1}_tas&pr_{compbase}.pdf') # {timeframe}



        


In [ ]:
        # plot difference plots:
        # ---------------------------------------------------
        
        
        # CALCULATE DIFFERENCE:
        #obs_regdata_mean = np.mean(obs_reg_data, axis=0)
        #model_regdata_mean = np.mean(model_reg_data, axis=0)
        diff_mean = obs_regdata_mean - model_regdata_mean
        try:
            len(diff_mean.flat)
        except TypeError:
            len(diff_mean.data.flat)
            diff_mean = diff_mean.data
            
        # spatial mean of differences:
        diff_mean_mean = np.average(diff_mean, weights=area_weights)
        
        #print(f'Mean difference = {diff_mean_mean:.2f}{regdatatypes_units[regdata_type]}')
        #print(f'Highest difference = {np.max(diff_mean):.2f}{regdatatypes_units[regdata_type]}')
        #print(f'Lowest difference = {np.min(diff_mean):.2f}{regdatatypes_units[regdata_type]}')
        
        # calculate & plot p-value of means comparison:
        diff_mean_p = z_test_two_means(obs_reg_data,model_reg_data)
        #diff_mean_p = np.ma.masked_where(diff_mean_p < 0.05, diff_mean_p)
        diff_title= f'Prob (p-value) of similarity to ERA5 mean {regdata_type} field'
        diff_text = f'Mean difference: {diff_mean_mean:.2f} {regdatatypes_units[regdata_type]}'
        diff_plot = plot_diff_p_map(diff_mean_p, lons, lats, regdata_type, fixed_bounds, standard_title, diff_title, diff_text)
        savediffmeanas = f'output/{model}/meandifftoERA5_probabilityp_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        diff_plot.savefig(savediffmeanas)
        
        

        #'''

        # plot difference:
        #diff_title= f'Mean field of geopotential height at 500 hPa \n Difference ERA5 - {model} \n Mean difference: {np.mean(diff_mean):.2f} hPa'
        #diff_title= f'Mean field of {regdata_type} \n Difference ERA5 - {model} \n Mean difference: {np.mean(diff_mean):.2f} {regdatatypes_units[regdata_type]}'
        diff_title= f'Difference to ERA5 mean {regdata_type} field'
        diff_text = f'Mean difference: {diff_mean_mean:.2f} {regdatatypes_units[regdata_type]}'
        #diff_colorlevels=np.linspace(-100,100,11)
        diff_plot = plot_diff_map(diff_mean, lons, lats, regdata_type, fixed_bounds, standard_title, diff_title, diff_text)
        savediffmeanas = f'output/{model}/meandifftoERA5_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        diff_plot.savefig(savediffmeanas)

        # significant difference:
        diff_mean_masked = np.ma.masked_where(diff_mean_p > 0.05, diff_mean)
        diff_mean_masked_mean = np.average(diff_mean_masked, weights=area_weights)
        print('Singificant differences (<5% probability of similarity):')
        #try:
        print(f'{diff_mean_masked.count()} / {len(diff_mean.flat)} = {(diff_mean_masked.count() / len(diff_mean.flat))*100:.2f} %' )
        #except AttributeError:
            #print(f'{diff_mean_masked.count()} / {len(diff_mean.data.flat)} = {(diff_mean_masked.count() / len(diff_mean.data.flat))*100:.2f} %' )
        
        print(f'Mean difference = {diff_mean_masked_mean:.2f}{regdatatypes_units[regdata_type]}')
        print(f'Highest difference = {np.max(diff_mean_masked):.2f}{regdatatypes_units[regdata_type]}')
        print(f'Lowest difference = {np.min(diff_mean_masked):.2f}{regdatatypes_units[regdata_type]}')
        
        plotting_modes=['mean+diff','mean+era','diff+era']
        plotting_mode = plotting_modes[2]
        diff_text= f'Mean difference: {diff_mean_mean:.2f} {regdatatypes_units[regdata_type]}'
        mean_diff_plot = plot_mean_plus_diff(model_regdata_mean,diff_mean,obs_regdata_mean,lons,lats,plotting_mode,regdata_type,standard_title,diff_text)
        savediffmeanas = f'output/{model}/meanpattern_{plotting_mode}_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        mean_diff_plot.savefig(savediffmeanas)
        
        diff_text= f'Mean difference: {diff_mean_masked_mean:.2f} {regdatatypes_units[regdata_type]}'
        mean_diff_plot = plot_mean_plus_diff(model_regdata_mean,diff_mean_masked,obs_regdata_mean,lons,lats,plotting_mode,regdata_type,standard_title,diff_text,masked=True)
        savediffmeanas = f'output/{model}/meanpattern_{plotting_mode}_masked_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        mean_diff_plot.savefig(savediffmeanas)
        
        '''
        pcr_diff = obs_coefficients-model_coefficients
        diff_title= f'{regdata_type} regression \n Difference ERA5 - {model} \n Mean difference: {np.mean(pcr_diff):.2f} {regdatatypes_units[regdata_type]}'
        diff_plot = plot_4_diff_map(pcr_diff, lons, lats, model_eofs, regdata_type, fixed_bounds, diff_title, order)
        savediffplotas = f'output/{model}/PCR_difftoERA5_{regvar}_{timeframe}_{model}_{variant}_{season}{meantext}{extr_text}.pdf'
        diff_plot.savefig(savediffplotas)      
        '''



In [ ]:
GL_skillscore = plot_taylor_diagrams(compmetrics, regvar, models, regdata_type, area='Greenland')

GL_weighted_score=weighted_skillscore(GL_skillscore,models,timeframe,variants,season,meantext,regridded_text,
                                   regvar,regdata_type,obs_varfrac,area='Greenland')    


In [ ]:
wgts = np.sqrt(np.abs(np.cos(np.radians(lats))))[..., np.newaxis]
solver = Eof(model_zg_data, weights = wgts, center=True)

n = n_eofs
eofs = solver.eofs(neofs = n, eofscaling = 0)   
    # 0 : Un-scaled EOFs (default).
    # 1 : EOFs are divided by the square-root of their eigenvalues.
    # 2 : EOFs are multiplied by the square-root of their eigenvalues.

pcs = solver.pcs(pcscaling=1, npcs = n)        
    # pcscaling=1 : PCs are scaled to unit variance (divided by the square-root of their eigenvalue).
    # same effect as dividing by standard deviation


varfrac = solver.varianceFraction(n)
eigenvalues = solver.eigenvalues(n)

errors = solver.northTest(n)
errors_scaled = solver.northTest(n, vfscaled=True)

varfrac, eigenvalues, errors, errors_scaled

In [ ]:
eigenvalues/np.sum(eigenvalues), errors/np.sum(errors)

In [ ]:
print(model_varfrac, obs_varfrac)
(obs_varfrac - model_varfrac)/obs_varfrac

In [ ]:
for im in range(len(models)):

    model = models[im]

    variant = variants[im]
    model_varfrac = np.load(f'data/{model}/varfrac_4EOFs_{timeframe}_{model}_{variant}_{season}_{meantext}{regridded_text}.npy')

    model_varfrac= np.array([model_varfrac[0], model_varfrac[2], model_varfrac[1], model_varfrac[3]])
    print(model, ':', np.round((obs_varfrac - model_varfrac)/obs_varfrac,3))
    
        